In [16]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [17]:
data=pd.read_csv('movies_dataset.csv')

C:\Users\CARRILLO JARA\AppData\Local\Temp\ipykernel_14216\2153530261.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('movies_dataset.csv')


In [18]:
# Eliminamos las filas duplicadas en base a la columna 'title'
data = data.drop_duplicates(subset='title')  


In [19]:
# Calculamos el promedio de la columna 'vote_average' y lo guarda en la variable C
C = data['vote_average'].mean()  


In [20]:
# Calcula el percentil 90 de la columna 'vote_count' y lo guarda en la variable m
m = data['vote_count'].quantile(0.90)  

In [21]:
# Filtra las filas donde 'vote_count' es mayor o igual que el percentil 90 y actualiza el DataFrame
data = data.loc[data['vote_count'] >= m]  

In [22]:
# Define una función 'weighted_rating' que calcula un puntaje ponderado para cada fila
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (m + v) * C)


In [23]:
# Aplica la función 'weighted_rating' a cada fila y guarda los resultados en una nueva columna 'score'
data['score'] = data.apply(weighted_rating, axis=1)  

In [24]:
# Ordena el DataFrame en base a la columna 'score' en orden descendente
data = data.sort_values('score', ascending=False)  

In [25]:
# Crea una instancia del vectorizador TF-IDF con palabras clave en inglés
tfidf = TfidfVectorizer(stop_words='english')  

In [26]:
# Rellena los valores faltantes de la columna 'overview' con una cadena vacía
data['overview'] = data['overview'].fillna('')  

In [27]:
# Transforma los datos de la columna 'overview' en una matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(data['overview'])  


In [28]:
# Calcula la similitud del coseno entre las filas de la matriz TF-IDF
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  


In [29]:
# Reinicia los índices del DataFrame y modifica el DataFrame en su lugar
data.reset_index(drop=True, inplace=True)  


In [ ]:
# Crea una Serie que mapea los títulos de las películas a sus índices correspondientes en el DataFrame
index = pd.Series(data.index, index=data['title']).drop_duplicates()  


In [ ]:
data = data.drop_duplicates(subset = 'title')
C = data['vote_average'].mean()
m = data['vote_count'].quantile(0.90)
data = data.loc[data['vote_count'] >= m]
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)
data['score'] = data.apply(weighted_rating, axis=1)
data = data.sort_values('score', ascending = False)
tfidf = TfidfVectorizer(stop_words='english')
data['overview'] = data['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['overview'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
data.reset_index(drop = True, inplace = True)
index = pd.Series(data.index, index = data['title']).drop_duplicates()

In [ ]:
def recomendacion(titulo, cosine_sim = cosine_sim):    
    if titulo not in index:
        return "La película no se encuentra entre el 10% de las mejores películas. Intenta con una mejor!"

    idx = index[titulo]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    result = data['title'].iloc[movie_indices]
    return {"Te recomendamos estas peliculas" : list(result)}

In [ ]:
recomendacion('Avatar')